In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import os
import gzip
import numpy as np
from subprocess import call

from __future__ import division
import medleydb as mdb
import numpy as np
import scipy
from scipy import interpolate
import sox
print(librosa.__version__)

/home/js7561/dev/miniconda3/envs/py35hpc/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


0.5.0dev


/home/js7561/dev/miniconda3/envs/py35hpc/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
vocalSet = set(['female singer', 'male rapper', 'male singer', 'male speaker', 'vocalists'])

In [3]:
# CQT Parameters

params_CQT = {
    'sr':44100, 
    'hop_length':1024, 
    'fmin':None, #Minimum frequency. Defaults to C1 ~= 32.70 Hz
    'n_bins':288, 
    'bins_per_octave':36, 
    'tuning':None, #Tuning offset in fractions of a bin (cents).
    'filter_scale':1, #Filter scale factor. Small values (<1) use shorter windows for improved time resolution.
    'norm':1, #Type of norm to use for basis function normalization.
    'sparsity':0.01, #Sparsify the CQT basis by discarding up to sparsity fraction of the energy in each basis.
    }

In [4]:
mdb.AUDIO_PATH

'/scratch/js7561/datasets/MedleyDB/Audio'

In [54]:
# parameters
output_folder = '/scratch/js7561/datasets/MedleyDB_output/original/features/'

# multitrack generator
trackList = os.listdir(mdb.AUDIO_PATH)
mtrack_generator = mdb.load_multitracks(trackList)

counter = 0
for track in mtrack_generator:
    
    # printout
    print(counter, track.track_id)
    
    # avoid repeated computation
    cqtfolder = 'cqt{:d}_{:d}_{:d}_{:d}'.format(
        params_CQT['sr'], params_CQT['hop_length'], 
        params_CQT['n_bins']//params_CQT['bins_per_octave'],
        params_CQT['bins_per_octave'])
    cqtfilename = '{:s}_cqt.npy.gz'.format(track.track_id)
    cqtfullpath = os.path.join(output_folder, cqtfolder, cqtfilename)
    
    actfolder = 'vocal_activation{:d}_{:d}'.format(params_CQT['sr'], params_CQT['hop_length'])
    actfilename = '{:s}_vocalactivation.npy.gz'.format(track.track_id)
    actfullpath = os.path.join(output_folder, actfolder, actfilename)
    
    if not (os.path.isfile(cqtfullpath) and os.path.isfile(actfullpath)):
    
        # load audio
        x, sr = librosa.load(track.mix_path, sr=params_CQT['sr'])

        # compute and save log-CQT
        logcqt = librosa.logamplitude(np.abs(librosa.cqt(y=x, **params_CQT)))
        np.save(gzip.open(cqtfullpath, 'wb'), logcqt)

        # compute vocal activation curve
        vocalIdx = [idx for idx, inst in enumerate(track.stem_instruments) if inst in vocalSet]

        try:
            if len(vocalIdx) != 0:
                stem_activations = np.array(track.stem_activations)[:,1:]
                time = np.array(track.stem_activations)[:,0]
                vocal_activation = np.max(stem_activations[:,vocalIdx],1)

                # interpolation function for vocal activation curve
                # some issue with end point(off by a few ms) so fill with last value
                f = interpolate.interp1d(time, vocal_activation, kind='linear',
                                         bounds_error=False, fill_value=vocal_activation[-1])

                # vocal activation lables per frame
                specLabel = [f(i * params_CQT['hop_length'] / params_CQT['sr']) for i in range(logcqt.shape[1])]
                specLabel = np.array(specLabel)
            else:
                specLabel = np.zeros(logcqt.shape[1])

            # save label
            np.save(gzip.open(actfullpath, 'wb'), specLabel)
        except IndexError:
            print('MISSING ACTIVATION: {:s}'.format(track.track_id))

    counter += 1

0 MusicDelta_FreeJazz
1 TablaBreakbeatScience_CaptainSky
2 TablaBreakbeatScience_PhaseTransition
3 MusicDelta_GriegTrolltog
4 CroqueMadame_Oil
5 TablaBreakbeatScience_RockSteady
6 KarimDouaidy_Yatora
7 TheDistricts_Vermont
8 MusicDelta_Vivaldi
9 MusicDelta_Rockabilly
10 AlexanderRoss_VelvetCurtain
11 HopAlong_SisterCities
12 ClaraBerryAndWooldog_WaltzForMyVictims
13 MusicDelta_Country2
14 MusicDelta_Rock
15 MusicDelta_Beethoven
16 MusicDelta_ModalJazz
17 AvaLuna_Waterduct
18 MusicDelta_Zeppelin
19 MusicDelta_SpeedMetal
20 MusicDelta_InTheHalloftheMountainKing
21 Wolf_DieBekherte
22 MusicDelta_ChineseChaoZhou
23 Phoenix_ColliersDaughter
24 ChrisJacoby_PigsFoot
25 MatthewEntwistle_FairerHopes
26 TablaBreakbeatScience_Animoog
27 Phoenix_ElzicsFarewell
28 MatthewEntwistle_TheArch
29 KarimDouaidy_Hopscotch
30 LizNelson_Coldwar
31 MusicDelta_Hendrix
32 MusicDelta_Country1
33 AmarLal_SpringDay1
34 MusicDelta_Beatles
35 Mozart_DiesBildnis
36 ChrisJacoby_BoothShotLincoln
37 MatthewEntwistle_AnE